In [5]:
import pyart
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import cartopy.crs as ccrs
from pyproj import Proj, CRS, Transformer
import glob
import xarray as xr
from datetime import datetime, timedelta
import pandas as pd

In [2]:
# Date of the data
year = '2025'
month = '03'
day = '13'

# The path to the data directory
# The data is stored in the following directory structure
# /mnt/gws/data/tone-ico_doppler_scanning_cloud_radar/L0/Y2025/M03/D13/
# The year, month and day are used to construct the path
data='/gws/nopw/j04/tone_ico_gws/data/tone-ico_doppler_scanning_cloud_radar/L0/Y'+year+'/M'+month+'/D'+day+'/*PPI.LV1.NC'
file_list = sorted(glob.glob(data))  

In [16]:
# storing the scans
scans = {}

# iterating over all the hourly radar scans during the given day
for file in file_list[:1]:
    ds = xr.open_dataset(file)

    # central lat/lon
    trollLat = ds['GPSLat'].values
    trollLon = ds['GPSLon'].values

    # Stack all CZE variables along the range dimension
    cze_vars = ['C1ZE45', 'C2ZE45', 'C3ZE45', 'C4ZE45']
    data_list = [ds[var].values for var in cze_vars]
    range_list = [ds[var.replace('ZE45', 'Range')].values for var in cze_vars]

    # Concatenate data and ranges
    data_stacked = np.concatenate(data_list, axis=1)  # shape: (n_az, total_range)
    ranges_stacked = np.concatenate(range_list)       # shape: (total_range,)

    # Set all values below -30 to NaN
    data_stacked[data_stacked < -30] = np.nan

    # Generate synthetic azimuths (assuming evenly spaced)
    n_az = ds['Time'].size
    azimuths = np.linspace(0, 2 * np.pi, n_az, endpoint=False)

    # Create 2D grids for polar plot
    azimuths_2d, ranges_2d = np.meshgrid(azimuths, ranges_stacked, indexing='ij')

    # Create Cartesian grids
    X = ranges_2d * np.cos(azimuths_2d)   
    Y = ranges_2d * np.sin(azimuths_2d)   

    # Transform to generate lat/lon grid
    crs_centered = CRS.from_proj4(f"+proj=aeqd +lat_0={trollLat} +lon_0={trollLon} +datum=WGS84 +units=m +no_defs")
    crs_geo = CRS.from_epsg(4326)  # WGS84 lat/lon
    transformer = Transformer.from_crs(crs_centered, crs_geo, always_xy=True)
    lon, lat = transformer.transform(X, Y)

    # Dealing with dates
    base_time = pd.Timestamp("2001-01-01T00:00:00Z")
    raw_times = ds['Time'].values  # shape: (n_azimuth,)
    converted_times = base_time + pd.to_timedelta(raw_times, unit="s")

    reflectivity_da = xr.DataArray(
        data=data_stacked,
        dims=["azimuth", "range"],
        coords={
            "azimuth": azimuths,
            "range": ranges_stacked,
            "time": ("azimuth", converted_times)
        },
        name="reflectivity"
    )

    print(reflectivity_da)






<xarray.DataArray 'reflectivity' (azimuth: 106, range: 250)> Size: 106kB
array([[       nan,        nan, 0.00012369, ..., 0.04529182, 0.04644562,
        0.04376626],
       [       nan,        nan,        nan, ..., 0.05536205, 0.05901112,
        0.05363842],
       [       nan,        nan, 0.00294858, ..., 0.05577671, 0.05760406,
        0.0592706 ],
       ...,
       [       nan,        nan, 0.00190774, ..., 0.06961532, 0.06791825,
        0.06544094],
       [       nan,        nan,        nan, ..., 0.06996405, 0.06656076,
        0.06741497],
       [       nan,        nan,        nan, ..., 0.0699541 , 0.06769476,
        0.06814254]], shape=(106, 250), dtype=float32)
Coordinates:
  * azimuth  (azimuth) float64 848B 0.0 0.05928 0.1186 ... 6.105 6.165 6.224
  * range    (range) float32 1kB 59.62 119.2 178.9 ... 1.491e+04 1.497e+04
    time     (azimuth) datetime64[ns, UTC] 848B 2025-03-13T00:40:00+00:00 ......
